In [ ]:
import os
import json
import torch

# 最终求解目标是，在满足约束的情况下，让前N抽达成抽卡目标的概率尽可能地小，以分析最坏情况下的抽卡成本

CHARACTER_BASE_PROB = 0.007
CHARACTER_DEPTH = 80
CHARACTER_EXP = 1 / 0.0186

WEAPON_BASE_PROB = 0.007
WEAPON_DEPTH = 60
WEAPON_EXP = 1 / 0.0237

LOADING_DIR = "results_0+1"
SAVING_FILE_TEMPLATE = os.path.join(LOADING_DIR, "gacha_min_prob_%d.json")
OVERWRITE=False

os.makedirs(LOADING_DIR, exist_ok=True)

# 最终概率分布是Params[0],(1-Params[0])*Params[1],(1-Params[0])*(1-Params[1])*Params[2]...
def GetDistribution(params, device):
    probs = torch.zeros(params.size(0) + 1, device=device)
    probs[0] = params[0]
    temp_prob = 1 - params[0]
    for i in range(1, params.size(0)):
        probs[i] = temp_prob * params[i]
        temp_prob = temp_prob * (1 - params[i])
    probs[-1] = temp_prob
    return probs


# 产生一个M+N的张量，第i个元素表示刚好在第i次抽卡时，既抽中prob1也抽中prob2的概率
def CrossProbs(prob1, prob2, device):
    probs = torch.zeros(prob1.size(0) + prob2.size(0), device=device)
    for i in range(prob1.size(0)):
        probs[i + 1 : i + prob2.size(0) + 1] += prob1[i] * prob2
    return probs


# 每个奖池第一次抽中时都有50%概率无效，所以要把一次抽中和两次抽中的概率叠加在一起，才是该池最终抽中的概率
def RepeatProbs(prob, rate, device):
    cross_prob = CrossProbs(prob, prob, device) * rate
    cross_prob[: prob.size(0)] += prob * (1 - rate)
    return cross_prob


def GetMinFinalProb(args):
    n_limit, epochs, lr, device = args
    if not OVERWRITE and os.path.exists(SAVING_FILE_TEMPLATE % n_limit):
        print(f"Skipping n_limit={n_limit} as it already exists")
        return
    print(f"Starting to solve for n_limit={n_limit} on device {device}")

    # 定义在1~k抽均未抽中的情况下，第k+1抽抽中的概率
    character_params = torch.nn.Parameter(
        torch.randn(CHARACTER_DEPTH - 1, device=device)
    )
    weapon_params = torch.nn.Parameter(torch.randn(WEAPON_DEPTH - 1, device=device))

    # 准备执行梯度下降法
    optimizer = torch.optim.Adam([character_params, weapon_params], lr=lr)
    best_loss = float("inf")
    best_probs = None
    for epoch in range(epochs):
        optimizer.zero_grad()

        # 由于概率是非负的，所以使用sigmoid函数将参数映射到[0,1]区间，并且保证基础概率不低于CHARACTER_BASE_PROB
        character_probs = (
            torch.nn.functional.sigmoid(character_params) * (1 - CHARACTER_BASE_PROB)
            + CHARACTER_BASE_PROB
        )
        weapon_probs = (
            torch.nn.functional.sigmoid(weapon_params) * (1 - WEAPON_BASE_PROB)
            + WEAPON_BASE_PROB
        )

        # 计算每金所需抽数的概率分布
        character_dist = GetDistribution(character_probs, device=device)
        weapon_dist = GetDistribution(weapon_probs, device=device)

        # 计算期望损失
        character_expect = torch.sum(
            character_dist * torch.arange(1, CHARACTER_DEPTH + 1, device=device)
        )
        weapon_expect = torch.sum(
            weapon_dist * torch.arange(1, WEAPON_DEPTH + 1, device=device)
        )
        expect_loss = (character_expect - CHARACTER_EXP) ** 2 + (
            weapon_expect - WEAPON_EXP
        ) ** 2

        # 计算每UP所需抽数的概率分布
        character_final_dist = RepeatProbs(character_dist, 0.5, device=device)
        weapon_final_dist = RepeatProbs(weapon_dist, 0.5, device=device)
        final_dist = CrossProbs(character_final_dist, weapon_final_dist, device=device)

        # 计算最终概率损失，即前N抽内达成目标的概率-第N+1抽及之后达成目标的概率
        final_prob_until_n = torch.sum(final_dist[:n_limit])
        final_prob_since_n = torch.sum(final_dist[n_limit:])

        # 计算总损失
        loss = expect_loss + final_prob_until_n - final_prob_since_n
        if loss.item() < best_loss:
            best_loss = loss.item()
            best_probs = (character_probs.clone(), weapon_probs.clone())

        loss.backward()
        optimizer.step()
        if (epoch + 1) % 500 == 0:
            print(
                f"Epoch {epoch}: Loss={loss.item()} FinalProbLoss={final_prob_until_n}-{final_prob_since_n} CharacterExpect={character_expect} WeaponExpect={weapon_expect}"
            )
    with open(SAVING_FILE_TEMPLATE % n_limit, "w") as f:
        json.dump(
            {
                "loss": best_loss,
                "character_probs": best_probs[0].cpu().detach().numpy().tolist(),
                "weapon_probs": best_probs[1].cpu().detach().numpy().tolist(),
            },
            f,
        )
    return best_loss

In [ ]:
import multiprocessing

MIN_N_LIMIT = 2
MAX_N_LIMIT = 278
POOL_SIZE = 2

CUDA_DEVICE_COUNT = 2
EPOCHS = 3000
LR = 0.02

# 为每个进程池分配不同的GPU设备
pools = [multiprocessing.Pool(POOL_SIZE) for _ in range(CUDA_DEVICE_COUNT)]
targets = [
    list(range(MIN_N_LIMIT + i, MAX_N_LIMIT + 1, CUDA_DEVICE_COUNT))
    for i in range(CUDA_DEVICE_COUNT)
]

# 同时开启所有进程池
# 注意这里不能使用正常的同步map
def OnError(e):
    print(e)
results = []
for i in range(CUDA_DEVICE_COUNT):
    results.append(
        pools[i].map_async(
            GetMinFinalProb, [(j, EPOCHS, LR, f"cuda:{i}") for j in targets[i]], error_callback=OnError
        )
    )

# 等待所有进程池结束
for i in range(CUDA_DEVICE_COUNT):
    results[i].wait()

# 关闭所有进程池
for i in range(CUDA_DEVICE_COUNT):
    pools[i].close()
    pools[i].join()

In [ ]:
import pandas

DEVICE="cuda"
#重新加载所有结果，首先将低于1.01倍BASE_PROB的概率设置为BASE_PROB，然后重新计算最终概率
results = {}
for file in os.listdir(LOADING_DIR):
    with open(os.path.join(LOADING_DIR, file), "r") as f:
        data = json.load(f)
        character_probs = torch.tensor(data["character_probs"], device=DEVICE)
        weapon_probs = torch.tensor(data["weapon_probs"], device=DEVICE)
        character_probs[character_probs < 1.01 * CHARACTER_BASE_PROB] = CHARACTER_BASE_PROB
        weapon_probs[weapon_probs < 1.01 * WEAPON_BASE_PROB] = WEAPON_BASE_PROB
        character_dist = GetDistribution(character_probs, device=DEVICE)
        weapon_dist = GetDistribution(weapon_probs, device=DEVICE)
        character_final_dist = RepeatProbs(character_dist, 0.5, device=DEVICE)
        weapon_final_dist = RepeatProbs(weapon_dist, 0.5, device=DEVICE)
        final_dist = CrossProbs(character_final_dist, weapon_final_dist, device=DEVICE)
        n_limit = int(file.split("_")[3].split(".")[0])
        results[n_limit] = torch.sum(final_dist[:n_limit]).item()
results=pandas.Series(results)
results[1]=0
results[280]=1
results.sort_index(inplace=True)
results

In [ ]:
CRITICAL_PROBS=[0.5,0.6,0.7,0.8,0.9,0.95,0.98,0.99]
CRITICAL_N_LIMITS=[results[results>=prob].index[0] for prob in CRITICAL_PROBS]
#打印表格
pandas.DataFrame({"目标概率":CRITICAL_PROBS,"最低总抽数":CRITICAL_N_LIMITS})

In [ ]:
import matplotlib.pyplot

#set the environment variable KMP_DUPLICATE_LIB_OK=TRUE
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

#绘制最终概率随n_limit的变化
matplotlib.pyplot.rcParams["font.sans-serif"] = ["SimHei"]
matplotlib.pyplot.figure(figsize=(12, 8))
matplotlib.pyplot.xticks([i for i in range(0, 301, 20)])
matplotlib.pyplot.yticks([i / 10 for i in range(11)])
matplotlib.pyplot.xlabel("总抽数")
matplotlib.pyplot.ylabel("概率")
#添加标题
matplotlib.pyplot.title("达成0+1的理论最低概率分析")
matplotlib.pyplot.grid(True)
matplotlib.pyplot.plot(results.index, results.values)